In [ ]:
%pip install ultralytics

In [ ]:
import cv2
from ultralytics import YOLO
#import numpy as np
import matplotlib.pyplot as plt

In [ ]:
model = YOLO("best1.pt")

In [ ]:
# Path to your test image (change this to your own image path)
image_path = "10.jpg"

# Run detection
results = model(image_path, conf=0.05)  # confidence threshold 50%

# results[0] contains detections for the first (and only) image
results[0].show()   # OpenCV popup (if supported)

In [ ]:
# Path to your image
image_path = "test.jpg"

# Run detection
results = model(image_path, conf=0.25)

# Get detection results for first (and only) image
boxes = results[0].boxes
names = model.names  # class names (COCO dataset: 'person' is class 0)

# Filter only "person" detections
person_boxes = []
for box in boxes:
    cls = int(box.cls[0])
    if names[cls] == "person":
        person_boxes.append(box)

print(f"Detected {len(person_boxes)} person(s).")

In [ ]:
img = cv2.imread(image_path)

# Draw only person detections
for box in boxes:
    cls = int(box.cls[0])
    if names[cls] == "person":
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
        conf = float(box.conf[0])
        label = f"Person {conf:.2f}"

        cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv2.putText(img, label, (int(x1), int(y1) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

# Show in OpenCV window
cv2.imshow("Person Detection", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
%pip install  pytesseract

In [ ]:
import cv2
import pytesseract
from matplotlib import pyplot as plt

# --- OPTIONAL: If tesseract is not in PATH, set the path manually ---
# For Windows, uncomment and adjust:
# pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

def ocr_image(image_path):
    # Load the image
    img = cv2.imread(image_path)

    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply thresholding
    _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)

    # Morphological noise removal
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 1))
    cleaned = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)

    # OCR with Tesseract
    text = pytesseract.image_to_string(cleaned)

    # Show original and processed image inline (Jupyter)
    fig, axs = plt.subplots(1, 2, figsize=(12, 6))
    axs[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    axs[0].set_title("Original Image")
    axs[0].axis("off")

    axs[1].imshow(cleaned, cmap="gray")
    axs[1].set_title("Processed for OCR")
    axs[1].axis("off")

    plt.show()

    return text


# Example usage
text_output = ocr_image("sample_text.png")
print("Extracted Text:\n", text_output)